In [2]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

In [5]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [6]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [7]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-05-13 22:52:17,826 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2022-05-13 22:52:17,827 - mmcv - INFO - load model from: torchvision://resnet50
2022-05-13 22:52:17,827 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/jjw/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
24.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

88.4%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limi

In [8]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

228, data_time: 0.003, memory: 2099, loss_rpn_cls: 0.0195, loss_rpn_bbox: 0.0281, loss_cls: 0.1872, acc: 94.0088, loss_bbox: 0.1719, loss: 0.4068, grad_norm: 1.6283
2022-05-13 23:41:34,979 - mmdet - INFO - Epoch [9][1150/1221]	lr: 2.000e-03, eta: 0:16:10, time: 0.228, data_time: 0.003, memory: 2099, loss_rpn_cls: 0.0204, loss_rpn_bbox: 0.0207, loss_cls: 0.1779, acc: 94.5176, loss_bbox: 0.1553, loss: 0.3742, grad_norm: 1.6588
2022-05-13 23:41:46,439 - mmdet - INFO - Epoch [9][1200/1221]	lr: 2.000e-03, eta: 0:15:57, time: 0.229, data_time: 0.003, memory: 2099, loss_rpn_cls: 0.0245, loss_rpn_bbox: 0.0280, loss_cls: 0.2033, acc: 93.7275, loss_bbox: 0.1887, loss: 0.4444, grad_norm: 1.7989
2022-05-13 23:41:51,250 - mmdet - INFO - Saving checkpoint at 9 epochs
2022-05-13 23:42:05,603 - mmdet - INFO - Epoch [10][50/1221]	lr: 2.000e-03, eta: 0:15:36, time: 0.273, data_time: 0.047, memory: 2099, loss_rpn_cls: 0.0218, loss_rpn_bbox: 0.0266, loss_cls: 0.1948, acc: 93.8633, loss_bbox: 0.1836, loss: